In [1]:
import dotenv
import os

dotenv_path = "../.env"
dotenv.load_dotenv(dotenv_path)

True

## Basic api connectivity

In [2]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "Which NFL team won the Super Bowl in the 2010 season?"

In [3]:
prompt.format(question=question)

'Question: Which NFL team won the Super Bowl in the 2010 season?\n\nAnswer: '

In [11]:
from langchain import HuggingFaceHub, LLMChain
from langchain_community.llms import HuggingFaceEndpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5, token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

/Users/alexhasha/Library/Caches/pypoetry/virtualenvs/llm-learning-pi5fb6Qz-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/alexhasha/.cache/huggingface/token
Login successful
 The New Orleans Saints won the Super Bowl XLIV in the 2010 season. They defeated the Indianapolis Colts with a score of 31-28. Drew Brees was the quarterback for the Saints and he was named the Super Bowl MVP. The game was played on February 7, 2010 at the Mercedes-Benz Superdome in New Orleans, Louisiana.


In [12]:
print(llm_chain.run(question))

 The New Orleans Saints won the Super Bowl XLIV in the 2010 season. They defeated the Indianapolis Colts with a score of 31-28. Drew Brees was the quarterback for the Saints and he was named the Super Bowl MVP. The game was played on February 7, 2010 at the Mercedes-Benz Superdome in New Orleans, Louisiana.


In [13]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text=' The New Orleans Saints won the Super Bowl XLIV in the 2010 season. They defeated the Indianapolis Colts with a score of 31-28. Drew Brees was the quarterback for the Saints and he was named the Super Bowl MVP. The game was played on February 7, 2010 at the Mercedes-Benz Superdome in New Orleans, Louisiana.')], [Generation(text='193.04 centimeters. To convert inches to centimeters, you can use the conversion factor of 1 inch = 2.54 centimeters. So, 6 ft 4 inches = 6 * 12 inches + 4 inches = 76 inches = 76 * 2.54 cm = 193.04 cm.')], [Generation(text='12 people have walked on the moon as of now. They are: Neil Armstrong, Buzz Aldrin, Michael Collins (Apollo 11), Edwin "Buzz" Aldrin, Charles "Pete" Conrad, Alan Bean, Harrison Schmitt, James Irwin, David Scott, John Young, Charles Duke, and Eugene Cernan. So, there is no 12th person on the moon, but the list above includes all the people who have walked on the lunar surface.')], [Generation(text='0.

The instruct model is much more terse and concise, it just does what it is instructed to do.

gpt-3.5-turbo is chatty, it will talk to you beyond what is asked.

So, if you want a model to do something the instruct model will typically work better and use fewer tokens doing so.

If you want a model to interact with users in a natural, conversational manner the chat model will be much better.

In [6]:
from langchain_openai import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.01)
print(llm("Tell me which NFL team won the Super Bowl in 2010?"))



The New Orleans Saints won the Super Bowl in 2010.


## Document Extraction

In [3]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from enum import Enum


class EmissionsCategory(str, Enum):
    Buildings = 'B'
    Energy = 'E'
    Transportation = 'T'
    Waste = 'W'
    LandUse = 'L'


class Goal(BaseModel):
    """Information about a strategic planning Goal.

    Goals are broad, quantifiable outcomes necessary to meet emissions targets and resilience goals.
    """
    id: str = Field(description="Unique Identifier.  If a Goal ID is given in the text, use this.  Otherwise, the first letter should be emissions_category letter.")
    emissions_category: EmissionsCategory = Field(description="The category of emissions the goal is associated with")
    year: Optional[int] = Field(default=None, description="The year by which the goal should be achieved.", ge=2024, le=2050)
    description: str = Field(default=None, description="A description of the goal which should include a specific, measurable, and quantifiable outcome")


class Strategy(BaseModel):
    """Information about a strategic planning Strategy.

    Strategies define general approaches to make progress toward goals.
    They should be specific, but ned not necessarily be quantifiable.
    They are frequently explicitly labeled as a "Strategy" in the text.
    However, they are often described as "goals" with no associated quantifiable outcome.
    """
    id: str = Field(description="Unique Identifier.  If a Strategy ID is given in the text, use this.  Otherwise, the first letter should be emissions_category letter.")
    emissions_category: EmissionsCategory = Field(description="The category of emissions the strategy is associated with")
    related_goals: List[str] = Field(description="A list of goal ids that this strategy is related to")
    description: str = Field(default=None, description="A description of the strategy")

class Action(BaseModel):
    """Information about a strategic planning Action.

    Actions are specific, time-bound steps to implement strategies.
    """
    id: str = Field(description="Unique Identifier.  First category should be emissions_category letter.")
    emissions_category: EmissionsCategory = Field(description="The category of emissions the strategy is associated with")
    owner: Optional[str] = Field(default=None, description="The organization or individuals responsible for the action, if known")
    related_stragegies: List[str] = Field(description="A list of strategy ids that this strategy is related to")
    description: str = Field(default=None, description="A description of the action")


class Results(BaseModel):
    goals: List[Goal]
    strategies: List[Strategy]
    actions: List[Action]

import pandas as pd

def results_to_dataframes(results):
    goals_df = pd.DataFrame(results.goals)
    strategies_df = pd.DataFrame(results.strategies)
    actions_df = pd.DataFrame(results.actions)

    return goals_df, strategies_df, actions_df

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert municipal climate action planner "
            """Read the provided Climate Action Plan document and extract the following information described in the text:
            1. Emissions reductions Goals;
            2. Strategies to achieve the goals;
            3. Action items related to the Strategies; """
            "If the text does not describe a goal, strategy, or action, do not make up an answer."
            "Only extract relevant information from the text."
            "If the value of an attribute you are asked to extract is not present in the text, return null for the attribute's value."
            "The same text cannot be both a goal and a strategy.  Please distinguish them based on their definitions."
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

## First attempt with cheaper, smaller LLMs

In [15]:
from langchain_openai import ChatOpenAI

#llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Results)

In [4]:
text = """OUR HOMES AND BUSINESSES
Making our homes and buildings super-efficient and electrifying our
space and water heating and cooking are core to Melrose’s Net
Zero Action Plan. The City of Melrose commits to implementing
actions that advance the following strategies:
Strategy 1. Electrify fossil-fuel end uses.
Strategy 2. Maximize uptake of residential energy efficiency
and deep energy retrofits in existing buildings.
Strategy 3. Target energy efficiency retrofits of large multi-
family buildings.
Strategy 4. Adopt policies to Incentivize energy efficiency and
renewable energy in new construction and major
renovations.
Strategy 5. Lead by example with municipal buildings and
advocate for net zero building policies.
INFOGRAPHIC CREDIT: MASSACHUSETTS CLEAN ENERGY CENTER
Melrose’s Net Zero Action Plan
20
In Melrose, 60 percent of building-related emissions result from the use of home
heating fuels like natural gas and fuel oil. Businesses and industry in Melrose rely
less heavily on fuel oil, but the use of natural gas and fuel oil still contribute
another 14 percent of building-related emissions. Melrose’s older housing stock
means the vast majority of homes were built before modern insulation and air
sealing practices and can benefit from weatherization and energy retrofits.
Burning oil and natural gas to heat our homes and cook our food creates 66,421
metric tons of emissions every year in Melrose. It also creates pollution both inside
and outdoors. In fact, recent studies have shown that methane leaks from gas-
burning stoves is far worse than previously thought."""

In [16]:
runnable.invoke({"text": text})

Results(goals=[Goal(id='E1', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Electrify fossil-fuel end uses'), Goal(id='E2', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Maximize uptake of residential energy efficiency and deep energy retrofits in existing buildings'), Goal(id='E3', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Target energy efficiency retrofits of large multi-family buildings'), Goal(id='E4', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Adopt policies to incentivize energy efficiency and renewable energy in new construction and major renovations'), Goal(id='E5', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Lead by example with municipal buildings and advocate for net zero building policies')], stragegies=[Strategy(id='E1', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E1'], description='Electrify f

In [17]:
result = Out[16]

In [18]:
result.goals

[Goal(id='E1', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Electrify fossil-fuel end uses'),
 Goal(id='E2', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Maximize uptake of residential energy efficiency and deep energy retrofits in existing buildings'),
 Goal(id='E3', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Target energy efficiency retrofits of large multi-family buildings'),
 Goal(id='E4', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Adopt policies to incentivize energy efficiency and renewable energy in new construction and major renovations'),
 Goal(id='E5', emissions_category=<EmissionsCategory.Energy: 'E'>, year=None, description='Lead by example with municipal buildings and advocate for net zero building policies')]

In [19]:
result.actions

[Action(id='E1', emissions_category=<EmissionsCategory.Energy: 'E'>, owner=None, related_stragegies=['E1'], description='Implement actions to electrify fossil-fuel end uses'),
 Action(id='E2', emissions_category=<EmissionsCategory.Energy: 'E'>, owner=None, related_stragegies=['E2'], description='Implement actions to maximize uptake of residential energy efficiency and deep energy retrofits in existing buildings'),
 Action(id='E3', emissions_category=<EmissionsCategory.Energy: 'E'>, owner=None, related_stragegies=['E3'], description='Implement actions to target energy efficiency retrofits of large multi-family buildings'),
 Action(id='E4', emissions_category=<EmissionsCategory.Energy: 'E'>, owner=None, related_stragegies=['E4'], description='Implement actions to adopt policies to incentivize energy efficiency and renewable energy in new construction and major renovations'),
 Action(id='E5', emissions_category=<EmissionsCategory.Energy: 'E'>, owner=None, related_stragegies=['E5'], descri

In [22]:
result.stragegies

[Strategy(id='E1', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E1'], description='Electrify fossil-fuel end uses'),
 Strategy(id='E2', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E2'], description='Maximize uptake of residential energy efficiency and deep energy retrofits in existing buildings'),
 Strategy(id='E3', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E2'], description='Target energy efficiency retrofits of large multi-family buildings'),
 Strategy(id='E4', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E4'], description='Adopt policies to incentivize energy efficiency and renewable energy in new construction and major renovations'),
 Strategy(id='E5', emissions_category=<EmissionsCategory.Energy: 'E'>, related_goals=['E5'], description='Lead by example with municipal buildings and advocate for net zero building policies')]

## Second attempt with premier LLM (gpt-4)

In [10]:
from langchain_openai import ChatOpenAI

llm2 = ChatOpenAI(model="gpt-4-turbo", temperature=0)

runnable2 = prompt | llm2.with_structured_output(schema=Results)

In [12]:
result2 = runnable2.invoke({"text": text})

In [13]:
result2.goals

[Goal(id='B2030', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2030, description='Achieve net zero emissions from buildings by 2030.')]

In [15]:
result2.strategies

[Strategy(id='B1', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030'], description='Electrify fossil-fuel end uses.'),
 Strategy(id='B2', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030'], description='Maximize uptake of residential energy efficiency and deep energy retrofits in existing buildings.'),
 Strategy(id='B3', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030'], description='Target energy efficiency retrofits of large multi-family buildings.'),
 Strategy(id='B4', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030'], description='Adopt policies to incentivize energy efficiency and renewable energy in new construction and major renovations.'),
 Strategy(id='B5', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030'], description='Lead by example with municipal buildings and advocate for net zero building policies.')]

In [16]:
result2.actions

[]

## Handling Documents

The following is from LangChain tutorial: https://python.langchain.com/v0.1/docs/use_cases/extraction/how_to/handle_files/

In [29]:
import requests

AMHERST_URL = "https://secureservercdn.net/50.62.195.83/env.320.myftpupload.com/wp-content/uploads/2021/09/Amherst_7.a.-CAARP-Final_061721-2.pdf"
URL = "https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf"

response = requests.get(URL)
data = response.content
data[:20]

b'%PDF-1.5\r%\xe2\xe3\xcf\xd3\r\n106 '

In [60]:
from langchain_community.document_loaders import PyMuPDFLoader

In [49]:
loader = PyMuPDFLoader(URL)

In [50]:
data = loader.load_and_split()

In [51]:
len(data)

13

In [53]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(data, OpenAIEmbeddings())

In [62]:
docs = faiss_index.similarity_search("Goals")
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

4: 4 | P a g e  
system or vehicle, we envision a climate program that helps to reduce the time, effort, and uncertainty in 
finding electrically sourced heat pumps and EVs . We envision a plan to help people find vetted, quality 
energy efficiency services; to provide guidance to climate‐benefitting and healthy food choices or waste 
management  practices;  and  to  easily  access  incentives  offered  by  our  utilities,  state,  and  Federal 
government.  As an example, Wayland is moving forward with creating economic and green sources of 
electricity for all town residents (Wayland Community Choice Electricity), as voted in 2021 Town Meeting.   
Supporting these goals, we include a role for local grassroots organizations like Energize Wayland, which 
support residents’ climate‐related decisions ‐ by providing coaching, vetting, and group‐buy arrangements 
‐  which  make  climate‐supportive  choices  less  risky,  less  expensive,  and  less  time  consuming.    As  an 
example, Ene

In [58]:
len(docs)

4

## Goal, Strategy, and Action Extraction from a PDF

In [13]:
from langchain_community.document_loaders import PyMuPDFLoader
URL = "https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf"

loader = PyMuPDFLoader(URL)

In [14]:
pages = loader.load()

In [15]:
pages[1]

Document(page_content='\xa0\n\xa0\nTOWN OF WAYLAND \n41 COCHITUATE ROAD \nWAYLAND, MASSACHUSETTS 01778 \n\xa0\n \n \nWayland Climate Action Mobilization Plan \nJune 2022 \n\xa0\n\xa0\n\xa0\n\xa0\nPreamble \n2 \nI. \nDecarbonize our Homes and Buildings \n5 \nII. \nMoving to Clean Energy Supply \n6 \nIII. \nTransportation - Driving Clean and Driving Less \n7 \nIV. Adaptation and Resilience \n8 \nV. \nNature-Based Actions to Mitigate Carbon \n9 \nVI. Food and Waste \n10 \nVII. Resident Engagement \n11 \n\xa0\n', metadata={'source': 'https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf', 'file_path': 'https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf', 'page': 1, 'total_pages': 12, 'format': 'PDF 1.5', 'title': 'Microsoft Word - Wayland climate action plan June 2022 for release v2.docx', 'author': 'jharp', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2.2', 'produc

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Results)

The first page is just a table of contents.  If the model is not hallucinating it should return nothing.

In [23]:
result = runnable.invoke({"text": pages[1].page_content})

In [24]:
result.goals

[]

In [25]:
result.strategies

[]

In [27]:
result.actions

[]

In [26]:
print(pages[1].page_content)

 
 
TOWN OF WAYLAND 
41 COCHITUATE ROAD 
WAYLAND, MASSACHUSETTS 01778 
 
 
 
Wayland Climate Action Mobilization Plan 
June 2022 
 
 
 
 
Preamble 
2 
I. 
Decarbonize our Homes and Buildings 
5 
II. 
Moving to Clean Energy Supply 
6 
III. 
Transportation - Driving Clean and Driving Less 
7 
IV. Adaptation and Resilience 
8 
V. 
Nature-Based Actions to Mitigate Carbon 
9 
VI. Food and Waste 
10 
VII. Resident Engagement 
11 
 



Page 5 includes multiple goals.  Let's see what happens with it.

In [28]:
results5 = runnable.invoke({"text": pages[5].page_content})

In [29]:
results5.goals

[Goal(id='B2030', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2030, description='By 2030, 50% of homes and commercial buildings to complete energy audits and thermal upgrades.'),
 Goal(id='B2050', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2050, description='By 2050, 100% of non-municipal buildings to have thermal upgrades.'),
 Goal(id='B2030-2', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2030, description='By 2030, 25% of non-municipal buildings use heat pumps to achieve at least 75% of their heating needs and all air conditioning needs.'),
 Goal(id='B2050-2', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2050, description='By 2050, 100% of non-municipal buildings have thermal upgrades and use heat pumps for heating and cooling.'),
 Goal(id='B2030-3', emissions_category=<EmissionsCategory.Buildings: 'B'>, year=2030, description='By 2030, 25% of municipal buildings to have implemented decarbonization strategies.'),
 Goal(i

In [30]:
results5.strategies

[Strategy(id='B1', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030', 'B2050', 'B2030-2', 'B2050-2', 'B2030-3', 'B2050-3'], description='Encourage residents and property owners to make green choices and lead by example by continuing to decarbonize our municipal buildings.'),
 Strategy(id='B2', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030', 'B2050', 'B2030-2', 'B2050-2', 'B2030-3', 'B2050-3'], description='Support coaching and outreach efforts, identify qualified vendors, and ensure residents understand their choices making full use of state and utility rebates.'),
 Strategy(id='B3', emissions_category=<EmissionsCategory.Buildings: 'B'>, related_goals=['B2030', 'B2050', 'B2030-2', 'B2050-2', 'B2030-3', 'B2050-3'], description='Pursue town regulations to limit or prevent new gas hook ups in new construction, as allowed by state law.')]

In [31]:
results5.actions

[Action(id='B1', emissions_category=<EmissionsCategory.Buildings: 'B'>, owner='Wayland', related_stragegies=['B1'], description='Transition municipal buildings to be carbon neutral (use no on-site fossil fuel and, where feasible, generate solar power on-site).'),
 Action(id='B2', emissions_category=<EmissionsCategory.Buildings: 'B'>, owner='Wayland', related_stragegies=['B1'], description='Reduce energy use in new construction and rehab. Ensure by 2023 that all new municipal buildings and substantial rehabilitations of existing municipal buildings meet high performing building standards to reduce energy use, decarbonize to use no on-site fossil fuels unless unique circumstance require such fuels, and incorporate climate sensitive design.'),
 Action(id='B3', emissions_category=<EmissionsCategory.Buildings: 'B'>, owner='Wayland', related_stragegies=['B1'], description='Implement decarbonization strategies for all existing municipal buildings taking advantage of heating/cooling system rep

In [32]:
pages[5].metadata

{'source': 'https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf',
 'file_path': 'https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf',
 'page': 5,
 'total_pages': 12,
 'format': 'PDF 1.5',
 'title': 'Microsoft Word - Wayland climate action plan June 2022 for release v2.docx',
 'author': 'jharp',
 'subject': '',
 'keywords': '',
 'creator': 'PScript5.dll Version 5.2.2',
 'producer': 'Acrobat Distiller 10.1.16 (Windows)',
 'creationDate': "D:20220630202538-04'00'",
 'modDate': "D:20220630202538-04'00'",
 'trapped': ''}

In [44]:
import pandas as pd

def results_to_goals_table(results, page, municipality):
    records = [
        {**g.dict(), "document_url": page.metadata["source"], "page": page.metadata["page"], "municipality": municipality}
        for g in results.goals
    ]
    return pd.DataFrame(records)

def results_to_strategies_table(results, page, municipality):
    records = [
        {**s.dict(), "document_url": page.metadata["source"], "page": page.metadata["page"], "municipality": municipality}
        for s in results.strategies
    ]
    return pd.DataFrame(records)

def results_to_actions_table(results, page, municipality):
    records = [
        {**a.dict(), "document_url": page.metadata["source"], "page": page.metadata["page"], "municipality": municipality}
        for a in results.actions
    ]
    return pd.DataFrame(records)

In [42]:
pd.set_option('display.max_colwidth', None)

In [39]:
results_to_goals_table(results5, pages[5], "Wayland")

,id,emissions_category,year,description,document_url,page,municipality
0,B2030,EmissionsCategory.Buildings,2030,"By 2030, 50% of homes and commercial buildings...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland
1,B2050,EmissionsCategory.Buildings,2050,"By 2050, 100% of non-municipal buildings to ha...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland
2,B2030-2,EmissionsCategory.Buildings,2030,"By 2030, 25% of non-municipal buildings use he...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland
3,B2050-2,EmissionsCategory.Buildings,2050,"By 2050, 100% of non-municipal buildings have ...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland
4,B2030-3,EmissionsCategory.Buildings,2030,"By 2030, 25% of municipal buildings to have im...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland
5,B2050-3,EmissionsCategory.Buildings,2050,"By 2050, 100% of municipal buildings be decarb...",https://www.wayland.ma.us/sites/g/files/vyhlif...,5,Wayland


In [43]:
results_to_strategies_table(results5, pages[5], "Wayland")

,id,emissions_category,related_goals,description,document_url,page,municipality
0,B1,EmissionsCategory.Buildings,"[B2030, B2050, B2030-2, B2050-2, B2030-3, B2050-3]",Encourage residents and property owners to make green choices and lead by example by continuing to decarbonize our municipal buildings.,https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland
1,B2,EmissionsCategory.Buildings,"[B2030, B2050, B2030-2, B2050-2, B2030-3, B2050-3]","Support coaching and outreach efforts, identify qualified vendors, and ensure residents understand their choices making full use of state and utility rebates.",https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland
2,B3,EmissionsCategory.Buildings,"[B2030, B2050, B2030-2, B2050-2, B2030-3, B2050-3]","Pursue town regulations to limit or prevent new gas hook ups in new construction, as allowed by state law.",https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland


In [45]:
results_to_actions_table(results5, pages[5], "Wayland")

,id,emissions_category,owner,related_stragegies,description,document_url,page,municipality
0,B1,EmissionsCategory.Buildings,Wayland,[B1],"Transition municipal buildings to be carbon neutral (use no on-site fossil fuel and, where feasible, generate solar power on-site).",https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland
1,B2,EmissionsCategory.Buildings,Wayland,[B1],"Reduce energy use in new construction and rehab. Ensure by 2023 that all new municipal buildings and substantial rehabilitations of existing municipal buildings meet high performing building standards to reduce energy use, decarbonize to use no on-site fossil fuels unless unique circumstance require such fuels, and incorporate climate sensitive design.",https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland
2,B3,EmissionsCategory.Buildings,Wayland,[B1],Implement decarbonization strategies for all existing municipal buildings taking advantage of heating/cooling system replacements and other opportunities.,https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland
3,B4,EmissionsCategory.Buildings,Wayland,[B3],"Advocate for effective state policies at the state level for significant renewable energy, decarbonization, and building energy efficiency standards and funding to support municipalities and residents in taking decarbonization efforts and achieve environmental justice goals.",https://www.wayland.ma.us/sites/g/files/vyhlif9231/f/uploads/wayland_climate_action_plan_june_2022_2.pdf,5,Wayland


### Cost estimate

Each page API call took about 3000 tokens. 

In [3]:
goals = [
    Goal(
        id="B1",
        emissions_category=EmissionsCategory.Buildings,
        year=2030,
        description="Reduce building-related emissions by 50% by 2030."
    ),
    Goal(
        id="E1",
        emissions_category=EmissionsCategory.Energy,
        year=2030,
        description="Transition to 100% renewable energy by 2030."
    ),
]

In [8]:
import pandas as pd
pd.DataFrame([g.dict() for g in goals])

,id,emissions_category,year,description
0,B1,EmissionsCategory.Buildings,2030,Reduce building-related emissions by 50% by 2030.
1,E1,EmissionsCategory.Energy,2030,Transition to 100% renewable energy by 2030.


In [7]:
goals[0].dict()

{'id': 'B1',
 'emissions_category': <EmissionsCategory.Buildings: 'B'>,
 'year': 2030,
 'description': 'Reduce building-related emissions by 50% by 2030.'}

### Enabling defaults to avoid validation errors

In [9]:
test_case_yaml = """
goals:
  - id: N1
    emissions_category: L
    description: Encourage residents, businesses, and organizations to maintain or increase tree cover, especially around buildings, to reduce energy usage and carbon emissions.
    context: Reducing greenhouse gas emissions doesn't just require technology. Nature-based solutions ask the Town, residents, businesses, and organizations to take actions to boost Wayland’s natural landscapes and resources to reduce carbon emissions.
  - id: N2
    emissions_category: L
    description: Encourage property owners and managers to minimize the use of lawns and re-landscape excess lawn areas to increase native plantings for better carbon sequestration.
    context: Minimize use of lawns, as they are the least effective landscaping for carbon sequestration. We encourage property owners and managers to right‐size lawn areas to actual uses and to re‐landscape excess lawn areas to increase native plantings.
  - id: N3
    emissions_category: L
    description: Encourage property owners to adopt organic lawn-keeping practices to reduce the need for watering, pesticides, and artificial fertilizers, and increase carbon sequestration in the soil.
    context: Adopt organic lawn-keeping practices. We encourage property owners to adopt these practices for mature lawn maintenance, including proper mowing, to reduce or eliminate the need for watering, pesticides, and artificial fertilizers; such practices also can increase carbon sequestration in the soil.
strategies:
  - id: N4
    emissions_category: L
    description: Strengthen tree removal regulations to require strict maintenance of tree cover consistent with proper maintenance of critical utilities and town infrastructure to sequester large amounts of carbon.
    context: "Town Actions: Strengthen tree removal regulations to require strict maintenance of tree cover consistent with proper maintenance of critical utilities and town infrastructure. Trees and forested landscapes sequester large amounts of carbon."
  - id: N5
    emissions_category: L
    description: Increase the progressivity of water rates to incentivize users to transition to native plantings and enhance carbon sequestration in the plantings and soil.
    context: "Town Actions: Increase the progressivity of water rates. Pumping and treatment of water require energy. Higher usage of water is associated with lawns and other landscaping using non-native plants. Progressive water rates provide an incentive for users to transition to native plantings to enhance carbon sequestration in the plantings and soil."
  - id: N6
    emissions_category: L
    description: Consider implementing lawn watering limits for mature lawns to conserve water resources.
    context: "Town Actions: Consider lawn watering limits for mature lawns."
  - id: N7
    emissions_category: L
    description: Foster availability of subsidized or free native plants to promote their use in landscaping for better carbon sequestration.
    context: "Town Actions: Foster availability of subsidized or free native plants."
"""


In [10]:
from yaml import safe_load

In [14]:
class EmissionsCategory(str, Enum):
    Buildings = "B"
    Energy = "E"
    Transportation = "T"
    Waste = "W"
    LandUse = "L"


class Goal(BaseModel):
    """Information about a strategic planning Goal.

    Goals are broad, quantifiable outcomes necessary to meet emissions targets and resilience goals.
    If the goal does not mention a quantitative target, it is likely a Strategy instead.
    """

    id: str = Field(
        description="Unique Identifier.  If a Goal ID is given in the text, use this.  Otherwise, the first letter should be emissions_category letter."
    )
    emissions_category: EmissionsCategory = Field(
        description="""
        The category of emissions the goal is associated with, e.g. Buildings, Transportation, Waste or Land Use, Energy.
        Select Energy only if no other more specific emission category is mentioned, or if the goal pertains to electricity.
        """
    )
    year: Optional[int] = Field(
        default=None,
        description="The year by which the goal should be achieved.",
        ge=2024,
        le=2050,
    )
    description: str = Field(
        description="A summary description of the goal which must include a measurable, quantitative outcome",
    )
    context: str = Field(description="Verbatim text from the provided document on which the Goal description is based")


class Strategy(BaseModel):
    """Information about a strategic planning Strategy.

    Strategies define general approaches to make progress toward goals.
    They should be specific, but need not necessarily be quantifiable.
    They are frequently explicitly labeled as a "Strategy" in the text.
    However, they are often described in the text as "Goals", but with no associated quantitative outcome mentioned.
    """

    id: str = Field(
        description="Unique Identifier.  If a Strategy ID is given in the text, use this.  Otherwise, the first letter should be emissions_category letter."
    )
    emissions_category: EmissionsCategory = Field(
        description="""
        The category of emissions the goal is associated with, e.g. Buildings, Transportation, Waste or Land Use, Energy.
        Select Energy only if no other more specific emission category is mentioned, or if the goal pertains to electricity.
        """
    )
    related_goals: Optional[List[str]] = Field(
        description="A list of goal ids that this strategy is related to"
    )
    description: str = Field(default=None, description="A description of the strategy")
    context: str = Field(description="Verbatim text from the provided document on which the Strategy description is based")


class Action(BaseModel):
    """Information about a strategic planning Action.

    Actions are specific, time-bound steps to implement strategies.
    """

    id: str = Field(
        description="Unique Identifier.  First category should be emissions_category letter."
    )
    emissions_category: EmissionsCategory = Field(
        description="The category of emissions the strategy is associated with"
    )
    owner: Optional[List[str]] = Field(
        default=None,
        description="The organization or individuals responsible for the action, if known",
    )
    related_stragegies: Optional[List[str]] = Field(
        default=None,
        description="A list of strategy ids that this action is related to"
    )
    description: str = Field(description="A summary description of the action")
    context: str = Field(description="Verbatim text from the provided document on which the Action description is based")

In [15]:
class Results(BaseModel):
    goals: List[Goal] = []
    strategies: List[Strategy] = []
    actions: List[Action] = []

In [16]:
results = Results(**safe_load(test_case_yaml))

In [19]:
results.strategies

[Strategy(id='N4', emissions_category=<EmissionsCategory.LandUse: 'L'>, related_goals=None, description='Strengthen tree removal regulations to require strict maintenance of tree cover consistent with proper maintenance of critical utilities and town infrastructure to sequester large amounts of carbon.', context='Town Actions: Strengthen tree removal regulations to require strict maintenance of tree cover consistent with proper maintenance of critical utilities and town infrastructure. Trees and forested landscapes sequester large amounts of carbon.'),
 Strategy(id='N5', emissions_category=<EmissionsCategory.LandUse: 'L'>, related_goals=None, description='Increase the progressivity of water rates to incentivize users to transition to native plantings and enhance carbon sequestration in the plantings and soil.', context='Town Actions: Increase the progressivity of water rates. Pumping and treatment of water require energy. Higher usage of water is associated with lawns and other landsca

In [22]:
results.dict()

{'goals': [{'id': 'N1',
   'emissions_category': <EmissionsCategory.LandUse: 'L'>,
   'year': None,
   'description': 'Encourage residents, businesses, and organizations to maintain or increase tree cover, especially around buildings, to reduce energy usage and carbon emissions.',
   'context': "Reducing greenhouse gas emissions doesn't just require technology. Nature-based solutions ask the Town, residents, businesses, and organizations to take actions to boost Wayland’s natural landscapes and resources to reduce carbon emissions."},
  {'id': 'N2',
   'emissions_category': <EmissionsCategory.LandUse: 'L'>,
   'year': None,
   'description': 'Encourage property owners and managers to minimize the use of lawns and re-landscape excess lawn areas to increase native plantings for better carbon sequestration.',
   'context': 'Minimize use of lawns, as they are the least effective landscaping for carbon sequestration. We encourage property owners and managers to right‐size lawn areas to actu

In [23]:
import uuid

In [34]:
str(uuid.uuid4())

'ba40d122-889b-49ee-9009-7a38b1a8c3cb'